#### Build a basic model monitoring flow on Vertex AI.
1. Deploy the endpoint in the GUI and set the model monitoring settings (with 1 hour granularity).
2. Run the predictions on the endpoint using a dataset different than the training. 
3. Review the output results in the GUI.

This notebook will handle the tasks in Step#2.

In [104]:
# Set parameters.
PROJECT_ID    = "bonibon"
PROJECT       = "991126484363"
LOCATION      = "us-central1"
ENDPOINT_ID   = "6201245580656640"
ENDPOINT_NAME = "projects/"+PROJECT_ID+"/locations/"+LOCATION+"/endpoints/"+ENDPOINT_ID
print(ENDPOINT_NAME)

projects/bonibon/locations/us-central1/endpoints/6201245580656640


In [ ]:
# Install packages and restart kernel.
#!pip3 install --user --upgrade --quiet kfp google-cloud-pipeline-components
#!pip3 install jsonlines

In [105]:
# Import packages.
import json
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.google.client import AIPlatformClient

In [106]:
# Create endpoint handler.
endpoint = aiplatform.Endpoint(endpoint_name= ENDPOINT_NAME)

In [107]:
# Generate a sample instance.
instance ={"amount": "157277.00", "amountError": "0.00", "destzeroFlag":"0", 
          "nameDest":"C214774813", "nameOrig": "C2108616891", "newbalanceDest": "688015.98",
          "newbalanceOrig":"0.00", "oldbalanceDest":"451217.28", "oldbalanceOrig":"109.00", 
          "origzeroFlag":"0","type":"TRANSFER"}

In [109]:
# Call the model and get the prediction.
response = endpoint.predict([instance])
print('API response: ', response)

API response:  Prediction(predictions=[{'scores': [0.9999849200248718, 1.502110626461217e-05], 'classes': ['0', '1']}], deployed_model_id='6485121890762358784', explanations=None)


#### Read a BigQuery table as jsonl file for endpoint prediction.
https://cloud.google.com/bigquery/docs/samples/bigquery-extract-table-json#bigquery_extract_table_json-python

In [85]:
# Convert the scoring table in BigQuery into jsonl format.

from google.cloud import bigquery
client = bigquery.Client()
bucket_name = 'bonibon-bucket/fraud/data'
project= "bonibon"
dataset_id = "finance"
table_id = "fraud_data_test_42"

destination_uri = "gs://{}/{}".format(bucket_name, "fraud_data_test_42_debug.json")
dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.job.ExtractJobConfig()
job_config.destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    job_config=job_config,
    # Location must match that of the source table.
    location="US",
)  # API request
extract_job.result()  # Waits for job to complete.

ExtractJob<project=bonibon, location=US, id=e9f803a6-c8d5-41e7-9df4-d9a0718278b5>

### Manual process in between: 
1. replace all ": with ":"
2. replace all ," with ","
3. replace all "" with "
4. replace all } with },
5. insert { "instances":[ at the beginning
6. delete , at the end
7. insert ]} at the end.

In [70]:
import gcsfs
import json

gcs_file_system = gcsfs.GCSFileSystem(project="bonibon")
gcs_json_path = "gs://bonibon-bucket/fraud/data/fraud_data_test_42.json"
with gcs_file_system.open(gcs_json_path) as f:
    json_dict = json.load(f)

In [111]:
json_dict["instances"][1]

{'type': 'CASH_OUT',
 'amount': '43248.02',
 'nameOrig': 'C1309678931',
 'nameDest': 'C190273303',
 'oldbalanceOrig': '0',
 'newbalanceOrig': '0',
 'oldbalanceDest': '1161575.08',
 'newbalanceDest': '1150434.65',
 'origzeroFlag': '1',
 'destzeroFlag': '0',
 'amountError': '-54388',
 'id': '375cad75-62d6-479f-8698-0f82d4594b35',
 'isFraud': '0'}

In [112]:
response = endpoint.predict(json_dict['instances'])
print('API response: ', response)

API response:  Prediction(predictions=[{'scores': [0.9999670386314392, 3.291038228780963e-05], 'classes': ['0', '1']}, {'classes': ['0', '1'], 'scores': [0.9999715089797974, 2.849611337296665e-05]}, {'classes': ['0', '1'], 'scores': [0.9999891519546509, 1.08322974483599e-05]}, {'classes': ['0', '1'], 'scores': [0.9999313354492188, 6.870587094454095e-05]}, {'scores': [0.9999747276306152, 2.524153751437552e-05], 'classes': ['0', '1']}, {'scores': [0.9998142719268799, 0.0001857527531683445], 'classes': ['0', '1']}, {'classes': ['0', '1'], 'scores': [0.9999818205833435, 1.817058000597171e-05]}, {'scores': [0.9999066591262817, 9.332695481134579e-05], 'classes': ['0', '1']}, {'classes': ['0', '1'], 'scores': [0.9999927878379822, 7.147853011701955e-06]}, {'classes': ['0', '1'], 'scores': [0.9999866485595703, 1.337242883892031e-05]}, {'classes': ['0', '1'], 'scores': [0.9999499320983887, 4.998077565687709e-05]}, {'classes': ['0', '1'], 'scores': [0.9999490976333618, 5.085906377644278e-05]}, {'

#### Get online predictions and explanations with AutoML model.
https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl#tabular

In [113]:
def predict_tabular_classification_sample(
    project: str, location: str, endpoint: str, instances: List[Dict],
):
    aiplatform.init(project=project, location=location)
    endpoint = aiplatform.Endpoint(endpoint)
    response = endpoint.predict(instances=instances)
    for prediction_ in response.predictions:
        print(prediction_)
predict_tabular_classification_sample(
    project= PROJECT_ID, location= LOCATION, endpoint=ENDPOINT_ID, instances=json_dict['instances'])

{'scores': [0.9999670386314392, 3.291038228780963e-05], 'classes': ['0', '1']}
{'scores': [0.9999715089797974, 2.849611337296665e-05], 'classes': ['0', '1']}
{'classes': ['0', '1'], 'scores': [0.9999891519546509, 1.08322974483599e-05]}
{'scores': [0.9999313354492188, 6.870587094454095e-05], 'classes': ['0', '1']}
{'scores': [0.9999747276306152, 2.524153751437552e-05], 'classes': ['0', '1']}
{'scores': [0.9998142719268799, 0.0001857527531683445], 'classes': ['0', '1']}
{'scores': [0.9999818205833435, 1.817058000597171e-05], 'classes': ['0', '1']}
{'scores': [0.9999066591262817, 9.332695481134579e-05], 'classes': ['0', '1']}
{'classes': ['0', '1'], 'scores': [0.9999927878379822, 7.147853011701955e-06]}
{'classes': ['0', '1'], 'scores': [0.9999866485595703, 1.337242883892031e-05]}
{'classes': ['0', '1'], 'scores': [0.9999499320983887, 4.998077565687709e-05]}
{'scores': [0.9999490976333618, 5.085906377644278e-05], 'classes': ['0', '1']}
{'classes': ['0', '1'], 'scores': [0.999993622303009,

In [102]:
def explain_tabular_sample(
    project: str, location: str, endpoint_id: str, instance_dict: Dict
):

    aiplatform.init(project=project, location=location)
    endpoint = aiplatform.Endpoint(endpoint_id)
    response = endpoint.explain(instances=instance_dict, parameters={})

    for explanation in response.explanations:
        print(" explanation")
        # Feature attributions.
        attributions = explanation.attributions
        for attribution in attributions:
            print("  attribution")
            print("   baseline_output_value:", attribution.baseline_output_value)
            print("   instance_output_value:", attribution.instance_output_value)
            print("   output_display_name:", attribution.output_display_name)
            print("   approximation_error:", attribution.approximation_error)
            print("   output_name:", attribution.output_name)
            output_index = attribution.output_index
            for output_index in output_index:
                print("   output_index:", output_index)

    for prediction in response.predictions:
        print(prediction)

explain_tabular_sample(
    project= PROJECT_ID, location= LOCATION, endpoint_id=ENDPOINT_ID, instance_dict=json_dict['instances'])

 explanation
  attribution
   baseline_output_value: 0.9999698996543884
   instance_output_value: 0.9999670386314392
   output_display_name: 0
   approximation_error: 0.004377420551748184
   output_name: scores
   output_index: 0
 explanation
  attribution
   baseline_output_value: 0.9999698996543884
   instance_output_value: 0.9999715089797974
   output_display_name: 0
   approximation_error: 0.004310389851002913
   output_name: scores
   output_index: 0
 explanation
  attribution
   baseline_output_value: 0.9999698996543884
   instance_output_value: 0.9999891519546509
   output_display_name: 0
   approximation_error: 0.003701794978196038
   output_name: scores
   output_index: 0
 explanation
  attribution
   baseline_output_value: 0.9999698996543884
   instance_output_value: 0.9999313354492188
   output_display_name: 0
   approximation_error: 0.005088619095662969
   output_name: scores
   output_index: 0
 explanation
  attribution
   baseline_output_value: 0.9999698996543884
   insta

##### End of file.